In [1]:
# 将规范的数组, 转化为树型结构
class TreeNode:
    def __init__(self, x):
        self.val = x
        self.left = None
        self.right = None

class Node:
    def __init__(self, val, children=[]):
        self.val = val
        self.children = children
        
def array2tree(arr):
    n = arr.pop(0)
    root = None if n is None else TreeNode(n)
    queue = [root]
    while arr and queue:
        node = queue.pop(0)
        if node is None:
            continue
        
        n1 = arr.pop(0)
        node.left = None if n1 is None else TreeNode(n1)
        queue.append(node.left)
        if arr:
            n2 = arr.pop(0)
            node.right = None if n2 is None else TreeNode(n2)
            queue.append(node.right)
    return root

In [17]:
"""
分别用递归和迭代方式实现二叉树先序,中序和后序遍历
"""
def preOrder(head):
    r = []
    def help(node):
        nonlocal r
        if not node:
            return None
        r.append(node.val)
        help(node.left)
        help(node.right)
    help(head)
    return r

def preOrder2(head):
    r, stack = [], [head]
    while stack:
        node = stack.pop()
        r.append(node.val)
        if node.right:
            stack.append(node.right)
        if node.left:
            stack.append(node.left)
    return r

def inOrder(head):
    r = []
    def help(node):
        nonlocal r
        if not node:
            return None
        help(node.left)
        r.append(node.val)
        help(node.right)
    help(head)
    return r

def inOrder2(head):
    r, stack, cur = [], [], head
    while cur or stack:
        while cur:
            stack.append(cur)
            cur = cur.left
        cur = stack.pop()
        r.append(cur.val)
        cur = cur.right
    return r

def postOrder(head):
    r = []
    def help(node):
        nonlocal r
        if not node:
            return None
        help(node.left)
        help(node.right)
        r.append(node.val)
    help(head)
    return r

def postOrder2(head):
    r = []
    s1, s2 = [head], []
    while s1:
        node = s1.pop()
        s2.append(node)
        if node.left:
            s1.append(node.left)
        if node.right:
            s1.append(node.right)
    while s2:
        r.append(s2.pop().val)
    return r

def postOrder3(head):
    """这里使用一个堆栈比较难!"""
    r, stack, cur = [], [head], None
    while stack:
        cur = stack[-1]
        if cur.left and head != cur.left and head != cur.right:
            stack.append(cur.left)
        elif cur.right and head != cur.right:
            stack.append(cur.right)
        else:
            r.append(stack.pop().val)
            head = cur
    return r
    
head = array2tree([1,2,3,4,5,6,7])
print(preOrder(head))
print(preOrder2(head))
print(inOrder(head))
print(inOrder2(head))
print(postOrder(head))
print(postOrder2(head))
print(postOrder3(head))

[1, 2, 4, 5, 3, 6, 7]
[1, 2, 4, 5, 3, 6, 7]
[4, 2, 5, 1, 6, 3, 7]
[4, 2, 5, 1, 6, 3, 7]
[4, 5, 2, 6, 7, 3, 1]
[4, 5, 2, 6, 7, 3, 1]
[4, 5, 2, 6, 7, 3, 1]


In [5]:
"""
二叉树的序列化和反序列化
"""
def serialByPre(head):
    if not head:
        return '#!'
    res = f'{head.val}!'
    res += serialByPre(head.left)
    res += serialByPre(head.right)
    return res

def reconByPre(s):
    arr = s.split('!')
    def help(stack):
        val = stack.pop(0)
        head = Node(int(val))
        head.left = help(stack)
        head.right = help(stack)
        return head
    return help(arr)

def serialByLevel(head):
    if not head:
        return '#!'
    res = f'{head.val}!'
    stack = [head]
    while stack:
        node = stack.pop(0)
        if node.left:
            res += f'{node.left.val}!'
            stack.append(node.left)
        else:
            res += '#!'
        if node.right:
            res += f'{node.right.val}!'
            stack.append(node.right)
        else:
            res += '#!'
    return res

def reconByLevel(s):
    arr = s.split('!')
    genNode = lambda a: None if a == '#' else Node(a)
    index, head, node = 0, genNode(arr[0]), None
    if not arr:
        return None
    stack = [head]
    while stack:
        node = stack.pop(0)
        node.left = genNode(stack[index + 1])
        node.right = genNode(stack[index + 2])
        index += 2
        if node.left:
            stack.append(node.left)
        if node.right:
            stack.append(node.right)
    return head

head = array2tree([1,2,3,4,5,6,7])
print(serialByPre(head))
print(serialByLevel(head))

1!2!4!#!#!5!#!#!3!6!#!#!7!#!#!
1!2!3!4!5!6!7!#!#!#!#!#!#!#!#!


In [10]:
"""
遍历二叉树的神级方法
题目: 给定一棵二叉树的头结点, 完成二叉树的先序,中序和后序遍历. 如果二叉树的节点数为N,
要求时间复杂度为O(N), 额外空间复杂度为O(1)
"""
def morrisIn(head):
    if not head:
        return []
    r = []
    cur1, cur2 = head, None
    while cur1:
        cur2 = cur1.left
        if cur2:
            # 存在右节点并且右节点不指向cur1(父节点)
            while cur2.right and cur2.right != cur1:
                cur2 = cur2.right
            if not cur2.right:
                cur2.right = cur1
                cur1 = cur1.left
                continue
            else:
                cur2.right = None
        r.append(cur1.val)
        cur1 = cur1.right
    return r

def morrisPre(head):
    if not head:
        return []
    r = []
    cur1, cur2 = head, None
    while cur1:
        cur2 = cur1.left
        if cur2:
            while cur2.right and cur2.right != cur1:
                cur2 = cur2.right
            if not cur2.right:
                cur2.right = cur1
                r.append(cur1.val)
                cur1 = cur1.left
                continue
            else:
                cur2.right = None
        else:
            r.append(cur1.val)
        cur1 = cur1.right
    return r

def morrisPos(head):
    if not head:
        return []
    
    def reverseEdge(head):
        pre, nex = None, None
        while head:
            nex = head.right
            head.right = pre
            pre = head
            head = nex
        return pre
    def help(head):
        nonlocal r
        tail = reverseEdge(head)
        cur = tail
        while cur:
            r.append(cur.val)
            cur = cur.right
        reverseEdge(tail)
        
    r = []
    cur1, cur2 = head, None
    while cur1:
        cur2 = cur1.left
        if cur2:
            while cur2.right and cur2.right != cur1:
                cur2 = cur2.right
            if not cur2.right:
                cur2.right = cur1
                cur1 = cur1.left
                continue
            else:
                cur2.right = None
                help(cur1.left)
        cur1 = cur1.right
    help(head)
    return r
    
    

head = array2tree([4,2,6,1,3,5,7])
print(morrisIn(head))
print(morrisPre(head))
print(morrisPos(head))

[1, 2, 3, 4, 5, 6, 7]
[4, 2, 1, 3, 6, 5, 7]
[1, 3, 2, 5, 7, 6, 4]


In [14]:
"""
在二叉树中找到累加和为指定值的最长路径长度
"""
def getMaxLength(head, _sum):
    def help(head, _sum, preSum, level, maxLen, d):
        if not head:
            return maxLen
        # 当前的节点总和
        curSum = preSum + head.val
        # 如果和不在字典中, 则写入到字典
        if curSum not in d:
            d[curSum] = level
        # 如果其差值在字典中, 说明到当前节点存在和为sum的情况, name就获取其最大值
        # 这里层数应该是当前层数 - 差值的层数
        if curSum - _sum in d:
            maxLen = max(level - d[curSum - _sum], maxLen)
        # 递归左右节点
        maxLen = help(head.left, _sum, curSum, level + 1, maxLen, d)
        maxLen = help(head.right, _sum, curSum, level + 1, maxLen, d)
        # 说明curSum遍历到此节点时候, 加上level的, 所以可以删除
        if level == d.get(curSum):
            del d[curSum]
        return maxLen
    
    d = {0: 0}
    return help(head, _sum, 0, 1, 0, d)

head = array2tree([-3,3,-9,1,0,2,1,None,None,1,6])
print(getMaxLength(head, 6))
print(getMaxLength(head, -9))

4
1


In [ ]:
"""
找到二叉树中的最大搜索二叉子树
"""